In [ ]:
import json
import os
import statistics
import matplotlib.pyplot as plt

# Preparing data

In [ ]:


# Initialize a dictionary to hold the data
data_dict = {}

query_type = 'path'

basepath = f'/home/tim/Datasets/query_optimization/{query_type}/'

savepath = '/home/tim/Datasets/plots/query_optimization/'

# Flag whether to save the plots
save_plots = True

# Iterate over each JSON file in the directory
for filename in os.listdir(basepath):
    
    # Only take queries with different plans
    if filename.startswith("no") and filename.endswith(".json"):
    # Only take queries with same plans
    #if not filename.startswith("no") and filename.endswith(".json"):
    # Take all queries
    #if filename.endswith('.json'):
    
        # Extract the number before 'star' from the filename
        number = int(filename.split('_')[-1].split(query_type)[0])
        
        # Initialize the lists if not already initialized
        if number not in data_dict:
            data_dict[number] = {'nLDE': [], 'nLDE_gnce': []}
        
        # Load the JSON content
        with open(f'{basepath}{filename}', 'r') as f:
            content = json.load(f)
        
        # Initialize temporary lists to store values for each query
        nLDE_tmp = []
        nLDE_gnce_tmp = []
        
        # Process each query in the JSON content
        for query_data in content:
            values = query_data['values']
            
            # Calculate the median of the first entries and append to nLDE_tmp
            first_entries = [x[0] for x in values]
            median_first = statistics.median(first_entries)
            nLDE_tmp.append(median_first)
            
            # Calculate the median of (second - third) entries and append to nLDE_gnce_tmp
            second_minus_third = [(x[1] - x[2]) for x in values]
            median_second_minus_third = statistics.median(second_minus_third)
            nLDE_gnce_tmp.append(median_second_minus_third)
        
        # Append the medians calculated for this JSON to the overall lists
        data_dict[number]['nLDE'].extend(nLDE_tmp)
        data_dict[number]['nLDE_gnce'].extend(nLDE_gnce_tmp)


# Scatter Plots

In [ ]:
import matplotlib.pyplot as plt

# Setting a font similar to the one in the attached image
#plt.rcParams['font.family'] = 'serif'
#plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']

# Create the scatter plot
all_x = []
all_y = []
for number, data in sorted(data_dict.items()):
    plt.scatter(data['nLDE'], data['nLDE_gnce'], marker='x', label=f"Number {number}")
    all_x.extend(data['nLDE'])
    all_y.extend(data['nLDE_gnce'])

# Add 45-degree line
min_val = min(min(all_x), min(all_y))
max_val = max(max(all_x), max(all_y))
plt.plot([min_val, max_val], [min_val, max_val], color='black', linewidth=0.5, linestyle='--')

# Add annotations for "Improvement" and "Degradation"
mid_x = (max_val - min_val) / 2 + min_val
mid_y_up = 3/4 * (max_val - min_val) + min_val
mid_y_down = 1/4 * (max_val - min_val) + min_val

plt.annotate('Improvement', xy=(mid_x, mid_y_down), xytext=(mid_x, mid_y_down),
             horizontalalignment='center', verticalalignment='center')
plt.annotate('Degradation', xy=(mid_x, mid_y_up), xytext=(mid_x, mid_y_up),
             horizontalalignment='center', verticalalignment='center')

# Set x and y limits
plt.xlim(min_val, max_val)
plt.ylim(min_val, max_val)


# Add labels and legend
plt.xlabel('nLDE')
plt.ylabel('nLDE + GNCE')
plt.legend(loc="upper left")

# Remove grid
plt.grid(False)

# Show the plot
plt.tight_layout() # Adjusts the layout to fit all elements

if save_plots:
    plt.savefig(f'{savepath}scatterplot_{query_type}.pdf')


# Boxplots

In [ ]:
import matplotlib.pyplot as plt

# Preparing data for boxplot
labels = []
nLDE_data = []
nLDE_gnce_data = []

for number, data in sorted(data_dict.items()):
    labels.append(f"{number} - nLDE")
    labels.append(f"{number} - nLDE + GNCE")
    nLDE_data.append(data['nLDE'])
    nLDE_gnce_data.append(data['nLDE_gnce'])

# Interleave the nLDE and nLDE_gnce data for correct ordering in the plot
plot_data = [item for sublist in zip(nLDE_data, nLDE_gnce_data) for item in sublist]

# Create the boxplot
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.boxplot(plot_data, vert=True, patch_artist=True, labels=labels)

# Coloring alternate boxes differently for distinction
colors = ['#D0E4F2', '#F2D0D0'] * (len(labels) // 2)  # Alternating colors
for patch, color in zip(ax.artists, colors):
    patch.set_facecolor(color)

# Set y label and title
ax.set_ylabel('Runtime')
ax.set_title('Boxplots of Runtime for nLDE and nLDE_gnce')

# Rotate x labels for better readability
plt.xticks(rotation=45, ha="right")

plt.tight_layout()

if save_plots:
    plt.savefig(f'{savepath}boxplot_{query_type}.pdf')



# Barplot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from cycler import cycler

# Extracting the mean values for nLDE and nLDE_gnce for each number
nLDE_medians = [np.mean(data['nLDE']) for number, data in sorted(data_dict.items())]
nLDE_gnce_medians = [np.mean(data['nLDE_gnce']) for number, data in sorted(data_dict.items())]

# Setting up the x-axis positions
numbers = sorted(data_dict.keys())
bar_width = 0.35
index = np.arange(len(numbers))

# Getting the colors from the colormap
colors = mpl.cm.tab10.colors

# Setting the hatch styles
bar_cycle = (cycler('hatch', ['//////','oooo', 'xxx', '**', 'OOO', '\\\\']))
styles = bar_cycle()

# Plotting the data
fig, ax = plt.subplots()
plt.rcParams['hatch.color'] = colors[1]
bar1 = ax.bar(index, nLDE_medians, bar_width, label='nLDE', color=colors[1], alpha=0.7, hatch=next(styles)['hatch'])
plt.rcParams['hatch.color'] = colors[0]
bar2 = ax.bar(index + bar_width, nLDE_gnce_medians, bar_width, label='nLDE + GNCE', alpha=0.7, color=colors[0], hatch=next(styles)['hatch'])

# Setting labels, title, and legend
ax.set_xlabel('Query Size (#tp)', fontsize=14)
ax.set_ylabel('Mean Runtime [s]', fontsize=14)
#ax.set_title('Barplots of Mean Runtime for nLDE and nLDE_gnce with Different Colors and Hatches')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(numbers)
ax.legend(fontsize=14)

plt.tight_layout()

if save_plots:
    plt.savefig(f'{savepath}barplot_{query_type}.pdf')


## Calculating Total Number of Queries

In [ ]:
n_queries = 0
for query_size in data_dict.values():
    n_queries += len(query_size['nLDE'])

In [ ]:
n_queries